RobôTAC

Gerador de documentos em escala.





Ordem das abas:

DADOS TAC, DADOS REC, Instruir, Notas, Prohealth, Sigfis, Dados, Boletim

# Bibliotecas

In [ ]:
!pip install gspread oauth2client
!pip install num2words
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install python-docx

from google.colab import auth, drive
from google.auth import default
import gspread



# Authenticate user and authorize access to Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

drive.mount('/content/drive')

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 3.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=d67f39aacd25a4e8b1137f90b1ad7d60cdb855e665967df4e8327d35846511ee
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 62.6 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.137.0
    Uninstalling google-api-python-client-2.137.0:
      Successfully uninstalled google-api-python-client-2.137.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 9.5 MB/s eta 0:00:00
Mounted at /content/drive


# TAC

In [ ]:
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_ALIGN_PARAGRAPH, WD_LINE_SPACING
from docx.shared import Pt, RGBColor, Inches
from docx.oxml.ns import qn
from datetime import datetime
import time
import pandas as pd
from num2words import num2words

start_time = time.time()


# Planilha
sheet_title = 'Dados TAC'
sheet = gc.open('Dados TAC').sheet1

# Convertendo planilha em Data Frame
tabela = pd.DataFrame(sheet.get_all_records())

for index, row in tabela.iterrows():
  doc = Document('/content/drive/MyDrive/docx_exemplo.docx')

  section = doc.sections[0]
  section.left_margin = Inches(0.8)
  section.right_margin = Inches(0.8)


  # Valor Monetário por extenso

  valor_texto = row["Valor"]
  valor_numerico = valor_texto.replace(",", ".").replace(".","")
  valor_numerico = valor_numerico.replace("$","")
  valor_numerico = int(valor_numerico)
  valor_numerico = valor_numerico / 100


  def valor_monetario_por_extenso(valor_numerico):

      # Separar a parte inteira e decimal do valor
      reais, centavos = divmod(valor_numerico, 1)
      centavos = round(centavos * 100)

      # Converter a parte inteira para extenso
      reais = num2words(reais, lang='pt_BR')

      # Converter a parte decimal para extenso
      centavos = num2words(centavos, lang='pt_BR')

      if centavos == 'zero':
        extenso_final = f'{reais} reais'
      else: # Assign a value to extenso_final when centavos is not zero
        extenso_final = f'{reais} reais e {centavos} centavos'

      return extenso_final


  # Informaões do Termo

  processo = row["Processo"]
  TAC = str(row['TAC'])
  ano_REC = str(row['Ano'])
  Fls = row['Fls']

  valor = row["Valor"]
  valor = 'R' + valor
  valor_monetario_por_extenso = valor_monetario_por_extenso(valor_numerico)
  valor_monetario_por_extenso = ' (' + valor_monetario_por_extenso + ')'

  periodo = row["Período"]
  empenho =  row["Empenho"]
  CNPJ = row["CNPJ"]
  empresa = row["Nome da empresa"]
  CPF = row["CPF"]
  endereco = row["endereco"]
  Nome_socio = row['Nome sócio']
  interessado = row['Interessado']
  servico = row['Objeto']

  # Data

  dia = str(datetime.now().strftime("%d"))
  mes_atual = datetime.now().month
  meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
          "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]
  nome_mes = meses[mes_atual - 1]
  ano = str(datetime.now().year)


  # Primeira página

  titlet = doc.add_heading(level=1)

  paragraph_format = titlet.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  run = titlet.add_run("TERMO DE AJUSTE DE CONTAS\n")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.underline = True
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  run = titlet.add_run(f"\nLivro: 002/{ano_REC}")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'


  run = titlet.add_run(f"\nTermo {TAC}/{ano_REC}")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  run = titlet.add_run(f"\nFls: {Fls}\n")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  run = titlet.add_run(f"\nTERMO DE AJUSTE DE CONTAS QUE ENTRE SI\nCELEBRAM O MUNICÍPIO DE DUQUE DE CAXIAS,\nATRAVÉS DA SECRETARIA MUNICIPAL DE SAÚDE E,\n {empresa}\nNA FORMA ABAIXO:")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  run.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  titlet.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run.font.color.rgb = RGBColor(0, 0, 0)


  run.font.all_caps = True
  run.bold = True


  # Data

  paragrafo = doc.add_paragraph()
  paragrafo.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = paragrafo.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  paragrafo4 = paragrafo.add_run('\nAos ')
  paragrafo4.font.size = Pt(11)
  paragrafo4.font.name = 'Arial'
  paragrafo4 = paragrafo.add_run(dia)
  paragrafo4.bold = True
  paragrafo4.font.size = Pt(11)
  paragrafo4.font.name = 'Arial'
  paragrafo4 = paragrafo.add_run(' de ')
  paragrafo4.bold = True
  paragrafo4.font.size = Pt(11)
  paragrafo4.font.name = 'Arial'
  paragrafo4 = paragrafo.add_run(nome_mes)
  paragrafo4.bold = True
  paragrafo4.font.size = Pt(11)
  paragrafo4.font.name = 'Arial'
  paragrafo4 = paragrafo.add_run(' de ')
  paragrafo4.bold = True
  paragrafo4.font.size = Pt(11)
  paragrafo4.font.name = 'Arial'
  paragrafo4 = paragrafo.add_run(ano)
  paragrafo4.bold = True
  paragrafo4.font.size = Pt(11)
  paragrafo4.font.name = 'Arial'

  #Texto

  paragrafo5 = paragrafo.add_run(", o Município de Duque de Caxias, inscrito no CNPJ sob o nº ")
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'

  paragrafo5 = paragrafo.add_run('29.138.328/0001-50')
  paragrafo5.bold = True
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'

  paragrafo5 = paragrafo.add_run(', com endereço na Alameda Dona Esmeralda nº 206, RJ, doravante denominado MUNICÍPIO, neste ato representado pelo Excelentíssimo Senhor Prefeito Wilson Miguel dos Reis, brasileiro, casado, portador da carteira de identidade nº 910.645.077, expedida pelo IFP/RJ e inscrito no CPF/ MF sob o nº 311.163.537-68, residente nesta cidade, que delega competência através da Lei Municipal nº 2.825 de 06 de Janeiro de 2017, ao ilmo. Secretária Municipal de Saúde, ')
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'


  paragrafo5 = paragrafo.add_run('Dra Célia Serrano da Silva')
  paragrafo5.bold = True
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'

  paragrafo5 = paragrafo.add_run(', brasileira, médica, portadora da carteira de identidade nº 2405860 expedida pelo SSP (PA) e inscrito no CPF sob o nº ')
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'
  paragrafo5 = paragrafo.add_run('392.515.002-15')
  paragrafo5.bold = True
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'
  paragrafo5 = paragrafo.add_run(', de outro lado, ')
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'

  # Empresa

  paragrafo2 = paragrafo.add_run(empresa)
  paragrafo2.bold = True
  paragrafo2.font.all_caps = True
  paragrafo2.font.size = Pt(11)
  paragrafo2.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(', doravante denominada CONTRATADA, pessoa jurídica de direito privado, com endereço, ')
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(endereco)
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(', devidamente inscrita no CNPJ sob nº ')
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(CNPJ)
  paragrafo3.bold = True
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'

  # Interessado

  paragrafo3 = paragrafo.add_run(f' neste ato representada pelo ')
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(Nome_socio)
  paragrafo3.bold = True
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(', devidamente inscrito no CPF nº ')
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(CPF)
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3.bold = True
  paragrafo3 = paragrafo.add_run(f', {interessado}')
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'


  # Processo

  paragrafo3 = paragrafo.add_run(', quando em conjunto denominadas as PARTES, tendo em vista o constante e decidido no Processo Administrativo de nº ')
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo2.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragrafo3 = paragrafo.add_run(processo)
  paragrafo3.bold = True
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'

  # Texto Fim

  paragrafo3 = paragrafo.add_run(', firmam o presente TERMO que se regerá incondicional e irrestritamente pela legislação específica Federal, mediante as seguintes cláusulas e condições:')
  paragrafo3.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'

  title = 1.5  # Espaçamento de 1.5 vezes o tamanho da fonte
  title = WD_LINE_SPACING.MULTIPLE
  title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

  # Centraliza o título
  title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

  doc.add_page_break()


  # Segunda Página


  # Cláusula Primeira

  title2 = doc.add_heading(level=1)
  title2.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run = title2.add_run("\n\nCLÁUSULA PRIMEIRA\n")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c1 = doc.add_paragraph()
  c1.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c1.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE
  c11 = c1.add_run('O presente TERMO tem por objetivo a liquidação e o pagamento da importância de ')
  c11.font.size = Pt(11)
  c11.font.name = 'Arial'
  c11 = c1.add_run(valor)
  c11.bold = True
  c11.font.size = Pt(11)
  c11.font.name = 'Arial'
  c11 = c1.add_run(valor_monetario_por_extenso)
  c11.bold = True
  c11.font.size = Pt(11)
  c11.font.name = 'Arial'
  c11 = c1.add_run('. Reconhecida a dívida pelo MUNICÍPIO em favor de ')
  c11.font.size = Pt(11)
  c11.font.name = 'Arial'
  c11 = c1.add_run(empresa)
  c11.bold = True
  c11.font.size = Pt(11)
  c11.font.name = 'Arial'
  c11 = c1.add_run('.')
  c11.font.size = Pt(11)
  c11.font.name = 'Arial'




  # Cláusula Segunda

  title3 = doc.add_heading(level=1)
  title3.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run = title3.add_run("CLÁUSULA SEGUNDA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c2 = doc.add_paragraph()
  c2.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c2.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c22 = c2.add_run('\nA ')
  c22.font.size = Pt(11)
  c22.font.name = 'Arial'
  c22 = c2.add_run(empresa)
  c22.bold = True
  c22.font.size = Pt(11)
  c22.font.name = 'Arial'
  c22 = c2.add_run(' concorda em receber a importância de ')
  c22.font.size = Pt(11)
  c22.font.name = 'Arial'
  c22 = c2.add_run(valor)
  c22.bold = True
  c22.font.size = Pt(11)
  c22.font.name = 'Arial'
  c22 = c2.add_run(valor_monetario_por_extenso)
  c22.bold = True
  c22.font.size = Pt(11)
  c22.font.name = 'Arial'
  c22 = c2.add_run('.')
  c22.bold = True
  c22.font.size = Pt(11)
  c22.font.name = 'Arial'


  # Cláusula Terceira

  title3 = doc.add_heading(level=1)
  title3.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  paragraph_format = title3.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  run = title3.add_run("CLÁUSULA TERCEIRA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c3 = doc.add_paragraph()
  c3.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c3.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c33 = c3.add_run('Com o recebimento da importância estipulada na Cláusula Segunda, a ')
  c33.font.size = Pt(11)
  c33.font.name = 'Arial'
  c33 = c3.add_run(empresa)
  c33.bold = True
  c33.font.size = Pt(11)
  c33.font.name = 'Arial'
  c33 = c3.add_run('. dá ao MUNICÍPIO Quitação plena, rasa, geral e irrevogável com relação a qualquer direito creditório originário do que ficou estipulado na Cláusula Primeira, eventuais juros devidos, correção monetária incidente sobre a importância paga quitada, reajustamento de preço, encargos sociais ou fiscais, ou quaisquer valores financeiros doutra forma vinculados ao ajustado na Cláusula Primeira.')
  c33.font.size = Pt(11)
  c33.font.name = 'Arial'

  # Cláusula Quarta

  title4 = doc.add_paragraph()
  title4.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run = title4.add_run("CLÁUSULA QUARTA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.bold = True
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c4 = doc.add_paragraph()
  c4.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c4.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c44 = c4.add_run(empenho)
  c44.font.size = Pt(11)
  c44.font.name = 'Arial'
  c44 = c4.add_run('.')
  c44.font.size = Pt(11)
  c44.font.name = 'Arial'


  # Cláusula Quinta

  title5 = doc.add_heading(level=1)
  title5.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  paragraph_format = title5.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  run = title5.add_run("CLÁUSULA QUINTA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c5 = doc.add_paragraph()
  c5.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c5.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c55 = c5.add_run('Dentro do prazo de 20 (vinte) dias a contar da assinatura, o presente TERMO deverá ser publicado em extrato, no Boletim Oficial do Município de Duque de Caxias, a expensas do MUNICÍPIO.')
  c55.font.size = Pt(11)
  c55.font.name = 'Arial'


  doc.add_page_break()


  # Terceira Página


  # Cláusula Sexta

  title6 = doc.add_heading(level=1)
  title6.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run = title6.add_run("\n\n\nCLÁUSULA SEXTA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c6 = doc.add_paragraph()
  c6.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c6.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c66 = c6.add_run('\nO MUNICÍPIO providenciará, até o 5º (quinto) dia útil seguinte ao de sua assinatura, o encaminhamento de cópia autenticada do presente instrumento ao Tribunal de Contas do Estado do Rio de Janeiro e à Controladoria Geral do Município.')
  c66.font.size = Pt(11)
  c66.font.name = 'Arial'

  # Cláusula Sétima

  title7 = doc.add_heading(level=1)
  title7.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run = title7.add_run("CLÁUSULA SÉTIMA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c7 = doc.add_paragraph()
  c7.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c7.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c77 = c7.add_run('\nO MUNICÍPIO não se responsabiliza por indenização de qualquer natureza em decorrência de atos ou fatos vinculados à fiscalização e controle da execução orçamentária e da administração financeira.')
  c77.font.size = Pt(11)
  c77.font.name = 'Arial'

  # Cláusula Oitava

  title8 = doc.add_paragraph()
  title8.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run = title8.add_run("CLÁUSULA OITAVA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.bold = True
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c8 = doc.add_paragraph()
  c8.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c8.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c88 = c8.add_run('Fica eleito para o Foro da Comarca do Município de Duque de Caxias, para dirimir quaisquer dúvidas ou litígios oriundos do presente TERMO.')
  c88.font.size = Pt(11)
  c88.font.name = 'Arial'



  # Conclusão

  conclusão = doc.add_paragraph()
  conclusão.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = conclusão.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  conclusão2 = conclusão.add_run('E, por estarem assim ajustados, é lavrado o presente ')
  conclusão2.font.size = Pt(11)
  conclusão2.font.name = 'Arial'
  conclusão2 = conclusão.add_run('TERMO DE AJUSTE DE CONTAS')
  conclusão2.font.size = Pt(11)
  conclusão2.font.name = 'Arial'
  conclusão2.bold = True
  conclusão2 = conclusão.add_run(', que vai assinado pelas partes em 2 (duas) vias de igual teor e validade.')
  conclusão2.font.size = Pt(11)
  conclusão2.font.name = 'Arial'

  # Cidade e Data

  cd = doc.add_paragraph()
  cd.alignment = WD_ALIGN_PARAGRAPH.RIGHT
  cd1 = cd.add_run('\nDuque de Caxias, ')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(dia)
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(' de ')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(nome_mes)
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(' de ')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(ano)
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run('.')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'

  # Secretária

  contratado = doc.add_paragraph()

  contratado1 = contratado.add_run("\n\n___________________________")
  contratado1.font.color.rgb = RGBColor(0, 0, 0)
  contratado1.font.size = Pt(11)
  contratado1.font.name = 'Arial'


  contratado1 = contratado.add_run("\nSecretária Municipal de Saúde")
  contratado1.font.color.rgb = RGBColor(0, 0, 0)
  contratado1.font.size = Pt(11)
  contratado1.font.name = 'Arial'

  contratado.alignment = WD_ALIGN_PARAGRAPH.CENTER


  # Contratado

  assinatura = doc.add_paragraph()

  assinatura1 = assinatura.add_run("\n\n___________________________")
  assinatura1.font.color.rgb = RGBColor(0, 0, 0)
  assinatura1.font.size = Pt(11)
  assinatura1.font.name = 'Arial'

  assinatura2 = assinatura.add_run("\nContratado ")
  assinatura2.font.color.rgb = RGBColor(0, 0, 0)
  assinatura2.font.size = Pt(11)
  assinatura2.font.name = 'Arial'
  assinatura.alignment = WD_ALIGN_PARAGRAPH.CENTER
  assinatura2.font.size = Pt(11)
  assinatura2.font.name = 'Arial'


  doc.add_page_break()


  # Quarta Página

# Extrato do Termo de Ajuste de Contas

  ext = doc.add_heading(level=1)
  ext1 = ext.add_run("\n\n\nEXTRATO DO TERMO DE AJUSTE DE CONTAS")
  ext1.font.color.rgb = RGBColor(0, 0, 0)
  ext1.font.underline = True
  ext1.font.size = Pt(11)
  ext1.font.name = 'Arial'
  ext.alignment = WD_ALIGN_PARAGRAPH.CENTER


# Espécie

  ext2 = doc.add_heading(level=1)
  ext2.alignment = WD_ALIGN_PARAGRAPH.LEFT
  paragraph_format = ext2.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  ext1 = ext2.add_run("ESPÉCIE:")
  ext1.font.color.rgb = RGBColor(0, 0, 0)
  ext1.font.size = Pt(11)
  ext1.font.name = 'Arial'


  especies = doc.add_paragraph()
  especies.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = especies.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  especies1 = especies.add_run(f'Termo de Ajuste de Contas nº ')
  especies1.font.size = Pt(11)
  especies1.font.name = 'Arial'

  especie1 = especies.add_run(f'{TAC}/{ano_REC}')
  especie1.bold = True
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'

  especie1 = especies.add_run(f', encartado às fls ')
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'
  especie1 = especies.add_run(Fls)
  especie1.bold = True
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'
  especie1 = especies.add_run(', especificado no ')
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'
  especie1 = especies.add_run(f'Livro nº 02/{ano_REC}/SMS')
  especie1.bold = True
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'
  especie1 = especies.add_run(', conforme procedimento administrativo nº ')
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'
  especie1 = especies.add_run(processo)
  especie1.bold = True
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'

  especie1 = especies.add_run('.')
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'



# Partes

  ext3 = doc.add_heading(level=1)

  ext33 = ext3.add_run("PARTES:")
  ext33.font.color.rgb = RGBColor(0, 0, 0)
  ext33.font.size = Pt(11)
  ext33.font.name = 'Arial'
  ext33.alignment = WD_ALIGN_PARAGRAPH.LEFT
  paragraph_format = ext3.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  parte = doc.add_paragraph()
  parte.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = parte.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  partes = parte.add_run('O Município de Duque de Caxias, por meio da Secretaria Municipal de Saúde e ')
  partes.font.size = Pt(11)
  partes.font.name = 'Arial'
  partes = parte.add_run(empresa)
  partes.bold = True
  partes.font.size = Pt(11)
  partes.font.name = 'Arial'
  partes = parte.add_run('.')



# Objeto

  obj = doc.add_heading(level=1)

  obj1 = obj.add_run("OBJETO:")
  obj1.font.color.rgb = RGBColor(0, 0, 0)
  obj1.font.size = Pt(11)
  obj1.font.name = 'Arial'
  obj.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = obj.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  objeto = doc.add_paragraph()
  objeto.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = objeto.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  objeto1 = objeto.add_run('O presente TERMO tem por objeto a liquidação e o pagamento da importância de ')
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(valor)
  objeto1.bold = True
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(valor_monetario_por_extenso)
  objeto1.bold = True
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(', reconhecida a dívida pelo MUNICÍPIO em favor de ')
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(empresa)
  objeto1.bold = True
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(' associados ao objeto de ')
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(servico)
  objeto1.bold = True
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(', referente ao período de ')
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(periodo)
  objeto1.bold = True
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(', para atender as necessidades da Secretaria Municipal de Saúde, em conformidade com as especificações constantes no Procedimento Administrativo nº ')
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(processo)
  objeto1.bold = True
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run('.')
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY



# Cidade e Data

  cd = doc.add_paragraph()
  cd.alignment = WD_ALIGN_PARAGRAPH.RIGHT
  cd1 = cd.add_run('\nDuque de Caxias, ')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(dia)
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(' de ')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(nome_mes)
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(' de ')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(ano)
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run('.')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'


# Secretaria
  assinatura = doc.add_paragraph()
  assinatura.alignment = WD_ALIGN_PARAGRAPH.CENTER
  paragraph_format = assinatura.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE


  assinatura1 = assinatura.add_run("\n\n\nCélia Serrano da Silva ")
  assinatura1.font.color.rgb = RGBColor(0, 0, 0)
  assinatura1.font.all_caps = True
  assinatura1.bold = True
  assinatura1.font.size = Pt(11)
  assinatura1.font.name = 'Arial'

  assinaturasec = assinatura.add_run("\nSecretária Municipal de Saúde")
  assinaturasec.font.color.rgb = RGBColor(0, 0, 0)
  assinaturasec.font.size = Pt(11)
  assinaturasec.font.name = 'Arial'

  assinaturasec = assinatura.add_run("\nMat. 23098-0")
  assinaturasec.font.color.rgb = RGBColor(0, 0, 0)
  assinaturasec.font.size = Pt(11)
  assinaturasec.font.name = 'Arial'


# Nome do arquivo

  nome_arquivo = processo.replace('/','.')
  nome_do_arquivo = str(TAC + ' - ' + nome_arquivo)


  end_time = time.time()



# Salva o documento
  doc.save(f'/content/drive/MyDrive/TAC E REC - 2024/TAC - 2024/TAC {nome_do_arquivo}.docx')
  print(f"TAC {TAC} gerado para processo nº {processo} com sucesso!")
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time} segundos")

TAC 238 gerado para processo nº 014/002627/2024 com sucesso!
TAC 239 gerado para processo nº 014/002039/2024 com sucesso!
Tempo de execução: 2.018174409866333 segundos


# REC

In [ ]:
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_ALIGN_PARAGRAPH, WD_LINE_SPACING
from docx.shared import Pt, RGBColor, Inches
from docx.oxml.ns import qn
from datetime import datetime
import time
import pandas as pd
from google.colab import auth
from google.auth import default
import gspread
from num2words import num2words

start_time = time.time()

# Authenticate user and authorize access to Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet by its title
sheet_title = 'Dados TAC'  # Replace with your actual Google Sheet title
sheet = gc.open('Dados TAC').get_worksheet(1)

# Convertendo planilha em Data Frame
tabela = pd.DataFrame(sheet.get_all_records())

for index, row in tabela.iterrows():
  doc = Document('/content/drive/MyDrive/docx_exemplo.docx')

  section = doc.sections[0]
  section.left_margin = Inches(0.8)
  section.right_margin = Inches(0.8)


  # Valor Monetário por extenso

  valor_texto = row["Valor"]
  valor_numerico = valor_texto.replace(",", ".").replace(".","")
  valor_numerico = valor_numerico.replace("$","")
  valor_numerico = int(valor_numerico)
  valor_numerico = valor_numerico / 100


  def valor_monetario_por_extenso(valor_numerico):

      # Separar a parte inteira e decimal do valor
      reais, centavos = divmod(valor_numerico, 1)
      centavos = round(centavos * 100)

      # Converter a parte inteira para extenso
      reais = num2words(reais, lang='pt_BR')

      # Converter a parte decimal para extenso
      centavos = num2words(centavos, lang='pt_BR')

      if centavos == 'zero':
        extenso_final = f'{reais} reais'
      else: # Assign a value to extenso_final when centavos is not zero
        extenso_final = f'{reais} reais e {centavos} centavos'

      return extenso_final


  # Informaões do Termo

  processo = row["Processo"]
  TAC = str(row['REC'])
  ano_REC = str(row['Ano'])
  Fls = row['Fls']

  valor = row["Valor"]
  valor = 'R' + valor
  valor_monetario_por_extenso = valor_monetario_por_extenso(valor_numerico)
  valor_monetario_por_extenso = ' (' + valor_monetario_por_extenso + ')'

  periodo = row["Período"]
  empenho =  row["Empenho"]
  CNPJ = row["CNPJ"]
  empresa = row["Nome da empresa"]
  CPF = row["CPF"]
  endereco = row["endereco"]
  Nome_socio = row['Nome sócio']
  interessado = row['Interessado']
  servico = row['Objeto']

  # Data

  dia = str(datetime.now().strftime("%d"))
  mes_atual = datetime.now().month
  meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
          "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]
  nome_mes = meses[mes_atual - 1]
  ano = str(datetime.now().year)


  # Primeira página

  titlet = doc.add_heading(level=1)

  paragraph_format = titlet.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  run = titlet.add_run("TERMO DE RECONHECIMENTO DE DÍVIDA\n")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.underline = True
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  run = titlet.add_run(f"\nLivro: 001/{ano_REC}")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'


  run = titlet.add_run(f"\nTermo {TAC}/{ano_REC}")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  run = titlet.add_run(f"\nFls: {Fls}\n")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  run = titlet.add_run(f"\nTERMO DE RECONHECIMENTO DE DÍVIDA QUE ENTRE SI\nCELEBRAM O MUNICÍPIO DE DUQUE DE CAXIAS,\nATRAVÉS DA SECRETARIA MUNICIPAL DE SAÚDE,\n {empresa} NA FORMA ABAIXO:")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  run.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  titlet.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run.font.color.rgb = RGBColor(0, 0, 0)


  run.font.all_caps = True
  run.bold = True


  # Data
  paragrafo = doc.add_paragraph()
  paragrafo.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = paragrafo.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  paragrafo4 = paragrafo.add_run('\nAos ')
  paragrafo4.font.size = Pt(11)
  paragrafo4.font.name = 'Arial'
  paragrafo4 = paragrafo.add_run(dia)
  paragrafo4.bold = True
  paragrafo4.font.size = Pt(11)
  paragrafo4.font.name = 'Arial'
  paragrafo4 = paragrafo.add_run(' de ')
  paragrafo4.bold = True
  paragrafo4.font.size = Pt(11)
  paragrafo4.font.name = 'Arial'
  paragrafo4 = paragrafo.add_run(nome_mes)
  paragrafo4.bold = True
  paragrafo4.font.size = Pt(11)
  paragrafo4.font.name = 'Arial'
  paragrafo4 = paragrafo.add_run(' de ')
  paragrafo4.bold = True
  paragrafo4.font.size = Pt(11)
  paragrafo4.font.name = 'Arial'
  paragrafo4 = paragrafo.add_run(ano)
  paragrafo4.bold = True
  paragrafo4.font.size = Pt(11)
  paragrafo4.font.name = 'Arial'

  #Texto
  paragrafo5 = paragrafo.add_run(", o Município de Duque de Caxias, inscrito no CNPJ sob o nº ")
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'

  paragrafo5 = paragrafo.add_run('29.138.328/0001-50')
  paragrafo5.bold = True
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'

  paragrafo5 = paragrafo.add_run(', com endereço na Alameda Dona Esmeralda nº 206, RJ, doravante denominado MUNICÍPIO, neste ato representado pelo Excelentíssimo Senhor Prefeito Wilson Miguel dos Reis, brasileiro, casado, portador da carteira de identidade nº 910.645.077, expedida pelo IFP/RJ e inscrito no CPF/ MF sob o nº 311.163.537-68, residente nesta cidade, que delega competência através da Lei Municipal nº 2.825 de 06 de Janeiro de 2017, ao ilmo. Secretária Municipal de Saúde, ')
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'


  paragrafo5 = paragrafo.add_run('Dra Célia Serrano da Silva')
  paragrafo5.bold = True
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'

  paragrafo5 = paragrafo.add_run(', brasileira, médica, portadora da carteira de identidade nº 2405860 expedida pelo SSP (PA) e inscrito no CPF sob o nº ')
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'
  paragrafo5 = paragrafo.add_run('392.515.002-15')
  paragrafo5.bold = True
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'
  paragrafo5 = paragrafo.add_run(', de outro lado, ')
  paragrafo5.font.size = Pt(11)
  paragrafo5.font.name = 'Arial'

  # Empresa
  paragrafo2 = paragrafo.add_run(empresa)
  paragrafo2.bold = True
  paragrafo2.font.all_caps = True
  paragrafo2.font.size = Pt(11)
  paragrafo2.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(', doravante denominada CONTRATADA, pessoa jurídica de direito privado, com endereço, ')
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(endereco)
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(', devidamente inscrita no CNPJ sob nº ')
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(CNPJ)
  paragrafo3.bold = True
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'

  # Interessado
  paragrafo3 = paragrafo.add_run(f' neste ato representada pelo ')
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(Nome_socio)
  paragrafo3.bold = True
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(', devidamente inscrito no CPF nº ')
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3 = paragrafo.add_run(CPF)
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo3.bold = True
  paragrafo3 = paragrafo.add_run(f', {interessado}')
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'


  # Processo
  paragrafo3 = paragrafo.add_run(', quando em conjunto denominadas as PARTES, tendo em vista o constante e decidido no Processo Administrativo de nº ')
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'
  paragrafo2.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragrafo3 = paragrafo.add_run(processo)
  paragrafo3.bold = True
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'

  # Texto Fim
  paragrafo3 = paragrafo.add_run(', firmam o presente TERMO que se regerá incondicional e irrestritamente pela legislação específica Federal, mediante as seguintes cláusulas e condições:')
  paragrafo3.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragrafo3.font.size = Pt(11)
  paragrafo3.font.name = 'Arial'

  title = 1.5  # Espaçamento de 1.5 vezes o tamanho da fonte
  title = WD_LINE_SPACING.MULTIPLE
  title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

  # Centraliza o título
  title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

  doc.add_page_break()


  # Segunda Página


  # Cláusula Primeira

  title2 = doc.add_heading(level=1)
  title2.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run = title2.add_run("\n\nCLÁUSULA PRIMEIRA\n")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c1 = doc.add_paragraph()
  c1.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c1.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE
  c11 = c1.add_run('O presente TERMO tem por objetivo a liquidação e o pagamento da importância de ')
  c11.font.size = Pt(11)
  c11.font.name = 'Arial'
  c11 = c1.add_run(valor)
  c11.bold = True
  c11.font.size = Pt(11)
  c11.font.name = 'Arial'
  c11 = c1.add_run(valor_monetario_por_extenso)
  c11.bold = True
  c11.font.size = Pt(11)
  c11.font.name = 'Arial'
  c11 = c1.add_run('. Reconhecida a dívida pelo MUNICÍPIO em favor de ')
  c11.font.size = Pt(11)
  c11.font.name = 'Arial'
  c11 = c1.add_run(empresa)
  c11.bold = True
  c11.font.size = Pt(11)
  c11.font.name = 'Arial'
  c11 = c1.add_run('.')
  c11.font.size = Pt(11)
  c11.font.name = 'Arial'




  # Cláusula Segunda

  title3 = doc.add_heading(level=1)
  title3.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run = title3.add_run("CLÁUSULA SEGUNDA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c2 = doc.add_paragraph()
  c2.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c2.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c22 = c2.add_run('\nA ')
  c22.font.size = Pt(11)
  c22.font.name = 'Arial'
  c22 = c2.add_run(empresa)
  c22.bold = True
  c22.font.size = Pt(11)
  c22.font.name = 'Arial'
  c22 = c2.add_run(' concorda em receber a importância de ')
  c22.font.size = Pt(11)
  c22.font.name = 'Arial'
  c22 = c2.add_run(valor)
  c22.bold = True
  c22.font.size = Pt(11)
  c22.font.name = 'Arial'
  c22 = c2.add_run(valor_monetario_por_extenso)
  c22.bold = True
  c22.font.size = Pt(11)
  c22.font.name = 'Arial'
  c22 = c2.add_run('.')
  c22.bold = True
  c22.font.size = Pt(11)
  c22.font.name = 'Arial'


  # Cláusula Terceira

  title3 = doc.add_heading(level=1)
  title3.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  paragraph_format = title3.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  run = title3.add_run("CLÁUSULA TERCEIRA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c3 = doc.add_paragraph()
  c3.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c3.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c33 = c3.add_run('Com o recebimento da importância estipulada na Cláusula Segunda, a ')
  c33.font.size = Pt(11)
  c33.font.name = 'Arial'
  c33 = c3.add_run(empresa)
  c33.bold = True
  c33.font.size = Pt(11)
  c33.font.name = 'Arial'
  c33 = c3.add_run('. dá ao MUNICÍPIO Quitação plena, rasa, geral e irrevogável com relação a qualquer direito creditório originário do que ficou estipulado na Cláusula Primeira, eventuais juros devidos, correção monetária incidente sobre a importância paga quitada, reajustamento de preço, encargos sociais ou fiscais, ou quaisquer valores financeiros doutra forma vinculados ao ajustado na Cláusula Primeira.')
  c33.font.size = Pt(11)
  c33.font.name = 'Arial'

  # Cláusula Quarta

  title4 = doc.add_paragraph()
  title4.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run = title4.add_run("CLÁUSULA QUARTA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.bold = True
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c4 = doc.add_paragraph()
  c4.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c4.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c44 = c4.add_run(empenho)
  c44.font.size = Pt(11)
  c44.font.name = 'Arial'
  c44 = c4.add_run('.')
  c44.font.size = Pt(11)
  c44.font.name = 'Arial'


  # Cláusula Quinta

  title5 = doc.add_heading(level=1)
  title5.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  paragraph_format = title5.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  run = title5.add_run("CLÁUSULA QUINTA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c5 = doc.add_paragraph()
  c5.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c5.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c55 = c5.add_run('Dentro do prazo de 20 (vinte) dias a contar da assinatura, o presente TERMO deverá ser publicado em extrato, no Boletim Oficial do Município de Duque de Caxias, a expensas do MUNICÍPIO.')
  c55.font.size = Pt(11)
  c55.font.name = 'Arial'


  doc.add_page_break()


  # Terceira Página


  # Cláusula Sexta

  title6 = doc.add_heading(level=1)
  title6.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run = title6.add_run("\n\n\nCLÁUSULA SEXTA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c6 = doc.add_paragraph()
  c6.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c6.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c66 = c6.add_run('\nO MUNICÍPIO providenciará, até o 5º (quinto) dia útil seguinte ao de sua assinatura, o encaminhamento de cópia autenticada do presente instrumento ao Tribunal de Contas do Estado do Rio de Janeiro e à Controladoria Geral do Município.')
  c66.font.size = Pt(11)
  c66.font.name = 'Arial'

  # Cláusula Sétima

  title7 = doc.add_heading(level=1)
  title7.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run = title7.add_run("CLÁUSULA SÉTIMA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c7 = doc.add_paragraph()
  c7.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c7.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c77 = c7.add_run('\nO MUNICÍPIO não se responsabiliza por indenização de qualquer natureza em decorrência de atos ou fatos vinculados à fiscalização e controle da execução orçamentária e da administração financeira.')
  c77.font.size = Pt(11)
  c77.font.name = 'Arial'

  # Cláusula Oitava

  title8 = doc.add_paragraph()
  title8.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  run = title8.add_run("CLÁUSULA OITAVA")
  run.font.color.rgb = RGBColor(0, 0, 0)
  run.bold = True
  run.font.size = Pt(11)
  run.font.name = 'Arial'

  c8 = doc.add_paragraph()
  c8.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = c8.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  c88 = c8.add_run('Fica eleito para o Foro da Comarca do Município de Duque de Caxias, para dirimir quaisquer dúvidas ou litígios oriundos do presente TERMO.')
  c88.font.size = Pt(11)
  c88.font.name = 'Arial'



  # Conclusão

  conclusão = doc.add_paragraph()
  conclusão.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = conclusão.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  conclusão2 = conclusão.add_run('E, por estarem assim ajustados, é lavrado o presente ')
  conclusão2.font.size = Pt(11)
  conclusão2.font.name = 'Arial'
  conclusão2 = conclusão.add_run('TERMO DE RECONHECIMENTO DE DÍVIDA')
  conclusão2.font.size = Pt(11)
  conclusão2.font.name = 'Arial'
  conclusão2.bold = True
  conclusão2 = conclusão.add_run(', que vai assinado pelas partes em 2 (duas) vias de igual teor e validade.')
  conclusão2.font.size = Pt(11)
  conclusão2.font.name = 'Arial'

  # Cidade e Data

  cd = doc.add_paragraph()
  cd.alignment = WD_ALIGN_PARAGRAPH.RIGHT
  cd1 = cd.add_run('\nDuque de Caxias, ')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(dia)
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(' de ')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(nome_mes)
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(' de ')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(ano)
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run('.')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'

  # Secretária

  contratado = doc.add_paragraph()

  contratado1 = contratado.add_run("\n\n___________________________")
  contratado1.font.color.rgb = RGBColor(0, 0, 0)
  contratado1.font.size = Pt(11)
  contratado1.font.name = 'Arial'


  contratado1 = contratado.add_run("\nSecretária Municipal de Saúde")
  contratado1.font.color.rgb = RGBColor(0, 0, 0)
  contratado1.font.size = Pt(11)
  contratado1.font.name = 'Arial'

  contratado.alignment = WD_ALIGN_PARAGRAPH.CENTER


  # Contratado

  assinatura = doc.add_paragraph()

  assinatura1 = assinatura.add_run("\n\n___________________________")
  assinatura1.font.color.rgb = RGBColor(0, 0, 0)
  assinatura1.font.size = Pt(11)
  assinatura1.font.name = 'Arial'

  assinatura2 = assinatura.add_run("\nContratado ")
  assinatura2.font.color.rgb = RGBColor(0, 0, 0)
  assinatura2.font.size = Pt(11)
  assinatura2.font.name = 'Arial'
  assinatura.alignment = WD_ALIGN_PARAGRAPH.CENTER
  assinatura2.font.size = Pt(11)
  assinatura2.font.name = 'Arial'


  doc.add_page_break()


  # Quarta Página

  # Extrato do Termo de Ajuste de Contas
  ext = doc.add_heading(level=1)
  ext1 = ext.add_run("\n\n\nEXTRATO DO TERMO DE RECONHECIMENTO DE DÍVIDA")
  ext1.font.color.rgb = RGBColor(0, 0, 0)
  ext1.font.underline = True
  ext1.font.size = Pt(11)
  ext1.font.name = 'Arial'
  ext.alignment = WD_ALIGN_PARAGRAPH.CENTER


  # Espécie
  ext2 = doc.add_heading(level=1)
  ext2.alignment = WD_ALIGN_PARAGRAPH.LEFT
  paragraph_format = ext2.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  ext1 = ext2.add_run("ESPÉCIE:")
  ext1.font.color.rgb = RGBColor(0, 0, 0)
  ext1.font.size = Pt(11)
  ext1.font.name = 'Arial'


  especies = doc.add_paragraph()
  especies.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = especies.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  especies1 = especies.add_run(f'Termo de Reconhecimento de Dívida nº ')
  especies1.font.size = Pt(11)
  especies1.font.name = 'Arial'

  especie1 = especies.add_run(f'{TAC}/{ano_REC}')
  especie1.bold = True
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'

  especie1 = especies.add_run(f', encartado às fls ')
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'
  especie1 = especies.add_run(Fls)
  especie1.bold = True
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'
  especie1 = especies.add_run(', especificado no ')
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'
  especie1 = especies.add_run(f'Livro nº 01/{ano_REC}/SMS')
  especie1.bold = True
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'
  especie1 = especies.add_run(', conforme procedimento administrativo nº ')
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'
  especie1 = especies.add_run(processo)
  especie1.bold = True
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'

  especie1 = especies.add_run('.')
  especie1.font.size = Pt(11)
  especie1.font.name = 'Arial'



  # Partes
  ext3 = doc.add_heading(level=1)

  ext33 = ext3.add_run("PARTES:")
  ext33.font.color.rgb = RGBColor(0, 0, 0)
  ext33.font.size = Pt(11)
  ext33.font.name = 'Arial'
  ext33.alignment = WD_ALIGN_PARAGRAPH.LEFT
  paragraph_format = ext3.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  parte = doc.add_paragraph()
  parte.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = parte.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  partes = parte.add_run('O Município de Duque de Caxias, por meio da Secretaria Municipal de Saúde e ')
  partes.font.size = Pt(11)
  partes.font.name = 'Arial'
  partes = parte.add_run(empresa)
  partes.bold = True
  partes.font.size = Pt(11)
  partes.font.name = 'Arial'
  partes = parte.add_run('.')



  # Objeto
  obj = doc.add_heading(level=1)

  obj1 = obj.add_run("OBJETO:")
  obj1.font.color.rgb = RGBColor(0, 0, 0)
  obj1.font.size = Pt(11)
  obj1.font.name = 'Arial'
  obj.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = obj.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  objeto = doc.add_paragraph()
  objeto.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = objeto.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  objeto1 = objeto.add_run('O presente TERMO tem por objeto a liquidação e o pagamento da importância de ')
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(valor)
  objeto1.bold = True
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(valor_monetario_por_extenso)
  objeto1.bold = True
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(', reconhecida a dívida pelo MUNICÍPIO em favor de ')
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(empresa)
  objeto1.bold = True
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(' associados ao objeto de ')
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(servico)
  objeto1.bold = True
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(', referente ao período de ')
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(periodo)
  objeto1.bold = True
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(', para atender as necessidades da Secretaria Municipal de Saúde, em conformidade com as especificações constantes no Procedimento Administrativo nº ')
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run(processo)
  objeto1.bold = True
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1 = objeto.add_run('.')
  objeto1.font.size = Pt(11)
  objeto1.font.name = 'Arial'
  objeto1.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY



  # Cidade e Data

  cd = doc.add_paragraph()
  cd.alignment = WD_ALIGN_PARAGRAPH.RIGHT
  cd1 = cd.add_run('\nDuque de Caxias, ')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(dia)
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(' de ')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(nome_mes)
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(' de ')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(ano)
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run('.')
  cd1.bold = True
  cd1.font.size = Pt(11)
  cd1.font.name = 'Arial'


  # Secretaria
  assinatura = doc.add_paragraph()
  assinatura.alignment = WD_ALIGN_PARAGRAPH.CENTER
  paragraph_format = assinatura.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE


  assinatura1 = assinatura.add_run("\n\n\nCélia Serrano da Silva ")
  assinatura1.font.color.rgb = RGBColor(0, 0, 0)
  assinatura1.font.all_caps = True
  assinatura1.bold = True
  assinatura1.font.size = Pt(11)
  assinatura1.font.name = 'Arial'

  assinaturasec = assinatura.add_run("\nSecretária Municipal de Saúde")
  assinaturasec.font.color.rgb = RGBColor(0, 0, 0)
  assinaturasec.font.size = Pt(11)
  assinaturasec.font.name = 'Arial'

  assinaturasec = assinatura.add_run("\nMat. 23098-0")
  assinaturasec.font.color.rgb = RGBColor(0, 0, 0)
  assinaturasec.font.size = Pt(11)
  assinaturasec.font.name = 'Arial'



  # Nome do arquivo

  nome_arquivo = processo.replace('/','.')
  nome_do_arquivo = str(TAC + ' - ' + nome_arquivo)


  end_time = time.time()




  # Salvamento
  doc.save(f'/content/drive/MyDrive/TAC E REC - 2024/REC - 2024/REC {nome_do_arquivo}.docx')
  print(f"REC {TAC} gerado para processo nº {processo} com sucesso!")

execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time} segundos")

REC 183 gerado para processo nº 014/002262/2024 com sucesso!
REC 184 gerado para processo nº 014/002263/2024 com sucesso!
Tempo de execução: 1.7193043231964111 segundos


# TERMO E AUTORIZO

In [ ]:

from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_ALIGN_PARAGRAPH, WD_LINE_SPACING
from docx.shared import Pt, RGBColor, Inches
from docx.oxml.ns import qn
from datetime import datetime
import time
import pandas as pd
from num2words import num2words

start_time = time.time()

# LISTA DE PROCESSOS PARA INSTRUÇÃO

sheet_title = 'Dados TAC'
instruir = gc.open('Dados TAC').get_worksheet(2).get_all_records()

sheet_title = 'Dados TAC'
notas = gc.open('Dados TAC').get_worksheet(3).get_all_records()


# DATA FRAMES
instruir = pd.DataFrame(instruir)
notas = pd.DataFrame(notas)






for index, row in instruir.iterrows():
  doc = Document('/content/drive/MyDrive/docx_exemplo.docx')

  section = doc.sections[0]
  section.left_margin = Inches(0.8)
  section.right_margin = Inches(0.8)


# Data

  dia = str(datetime.now().strftime("%d"))
  mes_atual = datetime.now().month
  meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
          "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]
  nome_mes = meses[mes_atual - 1]
  ano = str(datetime.now().year)



# Valor Monetário por extenso

  valor_texto = row["VALOR"]
  valor_numerico = valor_texto.replace(",", ".").replace(".","")
  valor_numerico = valor_numerico.replace("$","")
  valor_numerico = int(valor_numerico)
  valor_numerico = valor_numerico / 100


  def valor_monetario_por_extenso(valor_numerico):

      # Separar a parte inteira e decimal do valor
      reais, centavos = divmod(valor_numerico, 1)
      centavos = round(centavos * 100)

      # Converter a parte inteira para extenso
      reais = num2words(reais, lang='pt_BR')

      # Converter a parte decimal para extenso
      centavos = num2words(centavos, lang='pt_BR')

      if centavos == 'zero':
        extenso_final = f'{reais} reais'
      else: # Assign a value to extenso_final when centavos is not zero
        extenso_final = f'{reais} reais e {centavos} centavos'

      return extenso_final


# Informaões do Termo

  processo = str(row["PROCESSO"])
  processo_fl = processo + ' Fl'


  valor = valor_texto
  valor2 = valor.replace("$", "R$ ")


  valor_monetario_por_extenso = valor_monetario_por_extenso(valor_numerico)
  valor_monetario_por_extenso = ' (' + valor_monetario_por_extenso + ')'


# Notas Fiscais

  processo = row['PROCESSO']
  notas_processo = notas[notas['PROCESSO'] == processo]

  agrupados = notas.groupby('PROCESSO')['NOTAS'].apply(lambda x: ', '.join(map(str, x))).reset_index()
  lista_notas = agrupados.loc[agrupados['PROCESSO'] == processo, 'NOTAS'].to_string(index = False)


  tipo = instruir['DOC'].iloc[0]
  qtd_notas = len(lista_notas)


  if tipo == 'DNF' and qtd_notas > 1:
    num_notas = 'às DANFES'

  elif tipo == 'DNF' and qtd_notas == 1:
    num_notas = 'a DANFE'


  elif tipo == 'NF' and qtd_notas > 1:
    num_notas = 'às Notas Fiscais'

  elif  tipo == 'NF' and qtd_notas == 1:
    num_notas = 'a Nota Fiscal'

  else:
    num_notas = 'a fatura de locação'


# Data de emissão

  emissao = notas[notas['PROCESSO'] == processo]
  maior_data = emissao['EMISSÃO'].max()
  menor_data = emissao['EMISSÃO'].min()

  if maior_data == menor_data:

    data_emissao = maior_data

  else:
    data_emissao = str(menor_data) + ' a ' + str(maior_data)


# Informações

  cnpj = row["CNPJ"]
  periodo = row["PERÍODO"]
  empresa = row["EMPRESA"]
  objeto = row["OBJETO"]
  notas_fiscais = str(lista_notas)

  ano_atual = str(datetime.now().year)

  ano_referencia = periodo.split('/')
  ano_referencia = str(ano_referencia[-1])

  if ano_referencia == ano_atual:
    termo = 'TERMO DE AJUSTE DE CONTAS'

  else:
    termo = 'TERMO DE RECONHECIMENTO DE DÍVIDA'


# AUTORIZO


  num_processo = doc.add_paragraph('\n')
  num_processo.alignment = WD_ALIGN_PARAGRAPH.RIGHT
  paragraph_format = num_processo.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  paragrafo = num_processo.add_run(f'Processo n° {processo_fl}')
  paragrafo.bold = True
  paragrafo.font.size = Pt(11)
  paragrafo.font.name = 'Arial'

  paragrafo = doc.add_paragraph()
  paragrafo.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = paragrafo.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  paragrafo1 = paragrafo.add_run('              Nos termos do Decreto 8.641, de 02 de janeiro de 2024, ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run('AUTORIZO')
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(' o Pagamento do Administrativo acima mencionado no valor de ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(valor2)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(valor_monetario_por_extenso)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(f', referente {num_notas} n° ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(notas_fiscais)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(' de ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(data_emissao)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(', associado aos serviços de ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(objeto)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(' prestados no período de ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(periodo)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(', apresentada pela empresa ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(empresa)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run('\n	   O exame da documentação associada ao presente pleito, nos termos do decreto supracitado, no âmbito das responsabilidades desta Secretaria Municipal de Saúde, foi considerado ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run('SEM RESSALVAS.')
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'


  paragrafo.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = paragrafo.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE


# Atesto notas fiscais

  if len(notas_processo) > 1:

     paragrafo = doc.add_paragraph()
     paragrafo.paragraph_format.left_indent = Inches(1.5)

     paragrafo1 = paragrafo.add_run(f'1. Notas Fiscais atestadas por:\n')
     paragrafo1.font.size = Pt(11)
     paragrafo1.font.name = 'Arial'

     for index, row in notas_processo.iterrows():
      processo = row['PROCESSO']
      nota = row['NOTAS']
      fiscais = row['FISCAL NOTA']
      emissao = row['EMISSÃO']

      paragrafo1 = paragrafo.add_run(f'\n{nota}  {fiscais}')
      paragrafo1.bold = True
      paragrafo1.font.size = Pt(8)
      paragrafo1.font.name = 'Arial'


  else:
      for index, row in notas_processo.iterrows():
        processo = row['PROCESSO']
        nota = row['NOTAS']
        fiscais = row['FISCAL NOTA']
        emissao = row['EMISSÃO']

        num_notas2 = 'Nota Fiscal atestada'
        paragrafo = doc.add_paragraph()
        paragrafo.paragraph_format.left_indent = Inches(1.5)

        atesto_notas = paragrafo.add_run(f'1. Nota Fiscal {nota} atestada por {fiscais}')
        atesto_notas.font.size = Pt(12)
        atesto_notas.font.name = 'Arial'


# Cidade e Data

  cd = doc.add_paragraph()
  cd.alignment = WD_ALIGN_PARAGRAPH.RIGHT
  cd1 = cd.add_run('\nDuque de Caxias, ')
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(dia)
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(' de ')
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(nome_mes)
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(' de ')
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(ano)
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run('.')
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'


# Secretaria
  assinatura = doc.add_paragraph()
  assinatura.alignment = WD_ALIGN_PARAGRAPH.CENTER
  paragraph_format = assinatura.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE


  assinatura1 = assinatura.add_run("\n\n\nCélia Serrano da Silva ")
  assinatura1.font.color.rgb = RGBColor(0, 0, 0)
  assinatura1.bold = True
  assinatura1.font.size = Pt(12)
  assinatura1.font.name = 'Arial'

  assinaturasec = assinatura.add_run("\nSecretária Municipal de Saúde")
  assinaturasec.font.color.rgb = RGBColor(0, 0, 0)
  assinaturasec.font.size = Pt(12)
  assinaturasec.font.name = 'Arial'

  assinaturasec = assinatura.add_run("\nMat. 23098-0")
  assinaturasec.font.color.rgb = RGBColor(0, 0, 0)
  assinaturasec.font.size = Pt(12)
  assinaturasec.font.name = 'Arial'


# QUEBRA
  doc.add_page_break()


# TERMO

  num_processo = doc.add_paragraph('\n\n')
  num_processo.alignment = WD_ALIGN_PARAGRAPH.RIGHT
  paragraph_format = num_processo.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  paragrafo1 = num_processo.add_run(f'Processo n° {processo_fl}')
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'


  paragrafo = doc.add_paragraph()
  paragrafo.alignment = WD_ALIGN_PARAGRAPH.CENTER
  paragraph_format = paragrafo.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE


  paragrafo1 = paragrafo.add_run(termo)
  paragrafo1.font.color.rgb = RGBColor(0, 0, 0)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'


  paragrafo = doc.add_paragraph()
  paragrafo.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = paragrafo.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE


  paragrafo1 = paragrafo.add_run('             Fica reconhecida a dívida no valor de ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(valor2)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(valor_monetario_por_extenso)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(', associado à ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(objeto)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(' no período de ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(periodo)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(f', referente {num_notas} n° ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(notas_fiscais)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(' de ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(data_emissao)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(', encartada ao processo em epígrafe em favor de ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(empresa)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run('., inscrita no CNPJ sob o n.º ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(cnpj)
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run('.')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'


# Cidade e Data

  cd = doc.add_paragraph()
  cd.alignment = WD_ALIGN_PARAGRAPH.RIGHT
  cd1 = cd.add_run('\nDuque de Caxias, ')
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(dia)
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(' de ')
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(nome_mes)
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(' de ')
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run(ano)
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'
  cd1 = cd.add_run('.')
  cd1.bold = True
  cd1.font.size = Pt(12)
  cd1.font.name = 'Arial'


# Secretaria
  assinatura = doc.add_paragraph()
  assinatura.alignment = WD_ALIGN_PARAGRAPH.CENTER
  paragraph_format = assinatura.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE


  assinatura1 = assinatura.add_run("\n\n\nCélia Serrano da Silva ")
  assinatura1.font.color.rgb = RGBColor(0, 0, 0)
  assinatura1.bold = True
  assinatura1.font.size = Pt(12)
  assinatura1.font.name = 'Arial'

  assinaturasec = assinatura.add_run("\nSecretária Municipal de Saúde")
  assinaturasec.font.color.rgb = RGBColor(0, 0, 0)
  assinaturasec.font.size = Pt(12)
  assinaturasec.font.name = 'Arial'

  assinaturasec = assinatura.add_run("\nMat. 23098-0")
  assinaturasec.font.color.rgb = RGBColor(0, 0, 0)
  assinaturasec.font.size = Pt(12)
  assinaturasec.font.name = 'Arial'


# Nome do arquivo

  nome_arquivo = processo.replace('/','.')

  end_time = time.time()









# Salvamento
  doc.save(f'/content/drive/MyDrive/TERMOS E AUTORIZOS/{nome_arquivo}.docx')
  print(f"Autorizo para processo nº {processo} gerado com sucesso!")


execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time} segundos")

Autorizo para processo nº 014/002262/2024 gerado com sucesso!
Tempo de execução: 2.9950976371765137 segundos


# PROHEALTH

In [ ]:
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_ALIGN_PARAGRAPH, WD_LINE_SPACING
from docx.shared import Pt, RGBColor, Inches
from docx.oxml.ns import qn
from datetime import datetime
import time
import pandas as pd
from google.colab import auth, drive
from google.auth import default
import gspread
from num2words import num2words

start_time = time.time()

drive.mount('/content/drive')

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


sheet_title = 'Dados TAC'  # Replace with your actual Google Sheet title
Prohealth = gc.open('Dados TAC').get_worksheet(4)

tabela = pd.DataFrame(Prohealth.get_all_records())


for index, row in tabela.iterrows():

  doc = Document('/content/drive/MyDrive/docx_exemplo.docx')

  section = doc.sections[0]
  section.left_margin = Inches(0.8)
  section.right_margin = Inches(0.8)

  # Data

  dia = str(datetime.now().strftime("%d"))
  mes_atual = datetime.now().month
  meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
            "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]
  nome_mes = meses[mes_atual - 1]
  ano = str(datetime.now().year)


  # Valor Monetário por extenso

  valor_texto = row['VALOR']
  valor_numerico = valor_texto.replace(",", ".").replace(".","")
  valor_numerico = valor_numerico.replace("$","")
  valor_numerico = int(valor_numerico)
  valor_numerico = valor_numerico / 100

  def valor_monetario_por_extenso(valor_numerico):
      # Separar a parte inteira e decimal do valor
      reais, centavos = divmod(valor_numerico, 1)
      centavos = round(centavos * 100)

      # Converter a parte inteira para extenso
      reais = num2words(reais, lang='pt_BR')

      # Converter a parte decimal para extenso
      centavos = num2words(centavos, lang='pt_BR')

      if centavos == 'zero':
        extenso_final = f'{reais} reais'
      else: # Assign a value to extenso_final when centavos is not zero
        extenso_final = f'{reais} reais e {centavos} centavos'

      return extenso_final

  # Informações

  processo = str(row['PROCESSO'])
  processo_fl = processo + ' Fl'

  valor = valor_texto
  valor = 'R' + valor + ' '

  valor_monetario_por_extenso = valor_monetario_por_extenso(valor_numerico)
  valor_monetario_por_extenso = '('+ valor_monetario_por_extenso + ')'

  nota_fiscal1 = row['NOTA']
  nota_fiscal = str(nota_fiscal1) + ' de '
  emissão = row['EMISSÃO']

  periodo = row['MÊS']
  periodo = str(periodo) + '/2024'
  empresa = row['EMPRESA']
  objeto = row['OBJETO']
  nome = row['NOME']
  fiscais = row['FISCAIS']



  num_processo = doc.add_paragraph('\n')
  num_processo.alignment = WD_ALIGN_PARAGRAPH.RIGHT
  paragraph_format = num_processo.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE


  processo = num_processo.add_run(f'Processo n° {processo_fl}')
  processo.bold = True
  processo.font.size = Pt(12)
  processo.font.name = 'Arial'

  paragrafo = doc.add_paragraph()
  paragrafo.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = paragrafo.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  paragrafo1 = paragrafo.add_run('        Nos termos do Decreto 8.641 de 02 de janeiro de 2024, ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run('AUTORIZO')
  paragrafo1.bold = True
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo1 = paragrafo.add_run(' o pagamento do Administrativo acima mencionado no valor de ')
  paragrafo1.font.size = Pt(12)
  paragrafo1.font.name = 'Arial'

  paragrafo2 = paragrafo.add_run(valor)
  paragrafo2.bold = True
  paragrafo2.font.size = Pt(12)
  paragrafo2.font.name = 'Arial'

  paragrafo3 = paragrafo.add_run(valor_monetario_por_extenso)
  paragrafo3.bold = True
  paragrafo3.font.size = Pt(12)
  paragrafo3.font.name = 'Arial'

  paragrafo4 = paragrafo.add_run(', referente Nota Fiscal nº ')
  paragrafo4.font.size = Pt(12)
  paragrafo4.font.name = 'Arial'

  paragrafo5 = paragrafo.add_run(nota_fiscal)
  paragrafo5.bold = True
  paragrafo5.font.size = Pt(12)
  paragrafo5.font.name = 'Arial'

  paragrafo6 = paragrafo.add_run(emissão)
  paragrafo6.bold = True
  paragrafo6.font.size = Pt(12)
  paragrafo6.font.name = 'Arial'

  paragrafo7 = paragrafo.add_run(', associado aos serviços prestados no período de ')
  paragrafo7.font.size = Pt(12)
  paragrafo7.font.name  ='Arial'

  paragrafo8 = paragrafo.add_run(periodo)
  paragrafo8.bold = True
  paragrafo8.font.size = Pt(12)
  paragrafo8.font.name = 'Arial'

  paragrafo8 = paragrafo.add_run(', apresentada pela empresa ')
  paragrafo8.font.size = Pt(12)
  paragrafo8.font.name = 'Arial'

  paragrafo8 = paragrafo.add_run(empresa)
  paragrafo8.bold = True
  paragrafo8.font.size = Pt(12)
  paragrafo8.font.name  ='Arial'

  paragrafo8 = paragrafo.add_run('. Como objeto de ')
  paragrafo8.font.size = Pt(12)
  paragrafo8.font.name = 'Arial'

  paragrafo8 = paragrafo.add_run(objeto)
  paragrafo8.bold = True
  paragrafo8.font.size = Pt(12)
  paragrafo8.font.name = 'Arial'

  paragrafo8 = paragrafo.add_run(', lastreado no ')
  paragrafo8.font.size = Pt(12)
  paragrafo8.font.name = 'Arial'

  paragrafo8 = paragrafo.add_run('Termo nº 01-001/2022, T.A nº 03-004/2023 e T.A nº 03-002/2024.')
  paragrafo8.bold = True
  paragrafo8.font.size = Pt(12)
  paragrafo8.font.name = 'Arial'

  paragrafo8 = paragrafo.add_run('\n        O exame da documentação associada ao presente pleito, nos termos do decreto supracitado, no âmbito das responsabilidades desta Secretaria Municipal de Saúde, foi considerado ')
  paragrafo8.font.size = Pt(12)
  paragrafo8.font.name = 'Arial'

  paragrafo8 = paragrafo.add_run('SEM RESSALVAS.')
  paragrafo8.bold = True
  paragrafo8.font.size = Pt(12)
  paragrafo8.font.name = 'Arial'


  # Notas atestadas
  paragrafo = doc.add_paragraph()
  paragrafo.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
  paragraph_format = paragrafo.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  paragrafo.paragraph_format.left_indent = Inches(1.5)


  atesto = paragrafo.add_run(f'1. Nota Fiscal nº {nota_fiscal1} atestada por {fiscais}.')
  atesto.font.size = Pt(11)
  atesto.font.name = 'Arial'

  atesto = paragrafo.add_run(f'\n\n2. Relatório de fiscalização assinado por {fiscais}.')
  atesto.font.size = Pt(11)
  atesto.font.name = 'Arial'

  atesto = paragrafo.add_run()
  atesto.font.size = Pt(11)
  atesto.font.name = 'Arial'

  atesto = paragrafo.add_run()
  atesto.font.size = Pt(11)
  atesto.font.name = 'Arial'





  # Data

  # Cidade e Data

  data = doc.add_paragraph()
  data.alignment = WD_ALIGN_PARAGRAPH.RIGHT
  data1 = data.add_run('\nDuque de Caxias, ')
  data1.bold = True
  data1.font.size = Pt(11)
  data1.font.name = 'Arial'
  data1 = data.add_run(dia)
  data1.bold = True
  data1.font.size = Pt(11)
  data1.font.name = 'Arial'
  data1 = data.add_run(' de ')
  data1.bold = True
  data1.font.size = Pt(11)
  data1.font.name = 'Arial'
  data1 = data.add_run(nome_mes)
  data1.bold = True
  data1.font.size = Pt(11)
  data1.font.name = 'Arial'
  data1 = data.add_run(' de ')
  data1.bold = True
  data1.font.size = Pt(11)
  data1.font.name = 'Arial'
  data1 = data.add_run(ano)
  data1.bold = True
  data1.font.size = Pt(11)
  data1.font.name = 'Arial'
  data1 = data.add_run('.')
  data1.bold = True
  data1.font.size = Pt(11)
  data1.font.name = 'Arial'



  # Secretaria
  assinatura = doc.add_paragraph()
  assinatura.alignment = WD_ALIGN_PARAGRAPH.CENTER
  paragraph_format = assinatura.paragraph_format
  paragraph_format.line_spacing_rule = WD_LINE_SPACING.ONE_POINT_FIVE

  assinatura1 = assinatura.add_run("\n\n\nCélia Serrano da Silva ")
  assinatura1.font.color.rgb = RGBColor(0, 0, 0)
  assinatura1.bold = True
  assinatura1.font.size = Pt(11)
  assinatura1.font.name = 'Arial'

  assinaturasec = assinatura.add_run("\nSecretária Municipal de Saúde")
  assinaturasec.font.color.rgb = RGBColor(0, 0, 0)
  assinaturasec.font.size = Pt(11)
  assinaturasec.font.name = 'Arial'

  assinaturasec = assinatura.add_run("\nMat. 23098-0")
  assinaturasec.font.color.rgb = RGBColor(0, 0, 0)
  assinaturasec.font.size = Pt(11)
  assinaturasec.font.name = 'Arial'



  end_time = time.time()

  processo_planilha = str(row['PROCESSO'])
  processo = processo_planilha.replace('/','.')


  doc.save(f'/content/drive/MyDrive/PROHEALTH/{processo} - {nome}.docx')
  print(f"Autorizo para {nome} gerado com sucesso!")





execution_time = end_time - start_time
print(f'Tempo de execução: {execution_time} segundos')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Autorizo para PROHEALTH gerado com sucesso!
Autorizo para DELTAMED gerado com sucesso!
Autorizo para HERA gerado com sucesso!
Autorizo para MEDPRIME gerado com sucesso!
Autorizo para HYGEA gerado com sucesso!
Autorizo para PROHEALTH gerado com sucesso!
Autorizo para DELTAMED gerado com sucesso!
Autorizo para HERA gerado com sucesso!
Autorizo para MEDPRIME gerado com sucesso!
Autorizo para HYGEA gerado com sucesso!
Autorizo para PROHEALTH gerado com sucesso!
Autorizo para DELTAMED gerado com sucesso!
Autorizo para HERA gerado com sucesso!
Autorizo para MEDPRIME gerado com sucesso!
Autorizo para HYGEA gerado com sucesso!
Tempo de execução: 7.73420262336731 segundos


# Saída

In [ ]:
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_ALIGN_PARAGRAPH, WD_LINE_SPACING
from docx.shared import Pt, RGBColor, Inches
from docx.oxml.ns import qn
from datetime import datetime
import time
import pandas as pd
from google.colab import auth, drive
from google.auth import default
import gspread
from num2words import num2words

sheet_title = 'Dados TAC'
notas = gc.open('Dados TAC').get_worksheet(3).get_all_records()


# DATA FRAMES

notas = pd.DataFrame(notas)

num_notas

,PROCESSO,NOTAS,EMISSÃO,VALOR,FISCAL NOTA
0,014/001846/2023,2526,08/07/2024,"R$ 256,00","Dra. Célia Guerra, Mat. 37774-0 e Matheus Tole..."
1,014/001846/2023,2527,08/07/2024,"R$ 256,00","Dra. Célia Guerra, Mat. 37774-0 e Daniele A. O..."
2,014/001846/2023,2529,08/07/2024,"R$ 357,00","Marco Antonio Alves de Souza, Mat. 37630-2 e M..."
3,014/001846/2023,2563,08/07/2024,"R$ 583,00","Dra. Célia Guerra, Mat. 37774-0 e Marta Corrêa..."
4,014/000906/2024,4589,15/07/2024,"R$ 321,20","Dra. Célia Guerra, Mat. 37774-0 e Matheus Tole..."
...,...,...,...,...,...
67,014/002262/2024,16066-1,22/12/2023,"R$ 171.844,17","Dra. Célia Guerra, Mat. 37774-0 e Vinícius Sen..."
68,014/002263/2024,16065-1,22/12/2023,"R$ 169.922,57","Dra. Célia Guerra, Mat. 37774-0 e Vinícius Sen..."
69,014/002557/2024,109547,10/07/2024,"R$ 102.400,00","Valéria Simone de França, Mat. 41.689-4 e Wal..."
70,014/002557/2024,109990,17/07/2024,"R$ 9.600,00","Valéria Simone de França, Mat. 41.689-4 e Wal..."


In [ ]:
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_ALIGN_PARAGRAPH, WD_LINE_SPACING
from docx.shared import Pt, RGBColor, Inches
from docx.oxml.ns import qn
from datetime import datetime
import time
import pandas as pd
from num2words import num2words

start_time = time.time()

# LISTA DE PROCESSOS PARA INSTRUÇÃO

sheet_title = 'Dados TAC'
instruir = gc.open('Dados TAC').get_worksheet(2).get_all_records()

instruir = pd.DataFrame(instruir)


valor = instruir["VALOR"].iloc[0]
valor = valor.replace("$", "R$ ")








tipo = instruir['DOC'].iloc[0]
processo = instruir['PROCESSO']
qtd_notas = 2



if tipo == 'DNF' and qtd_notas > 1:
  num_notas = 'às DANFES'

elif tipo == 'DNF' and qtd_notas == 1:
  num_notas = 'a DANFE'


elif tipo == 'NF' and qtd_notas > 1:
  num_notas = 'às Notas Fiscais'

elif  tipo == 'NF' and qtd_notas == 1:
  num_notas = 'a Nota Fiscal'

else:
  num_notas = 'a fatura de locação'



valor

'R$ 171.844,17'

In [ ]:
  tipo = instruir['DOC'].iloc[0]
  qtd_notas = len(lista_notas)


  if tipo == 'DNF' and qtd_notas > 1:
    num_notas = 'às DANFES'

  elif tipo == 'DNF' and qtd_notas == 1:
    num_notas = 'a DANFE'


  elif tipo == 'NF' and qtd_notas > 1:
    num_notas = 'às Notas Fiscais'

  elif  tipo == 'NF' and qtd_notas == 1:
    num_notas = 'a Nota Fiscal'

  else:
    num_notas = 'a fatura de locação'

'014/002262/2024'

In [ ]:
lista_notas

NameError: name 'lista_notas' is not defined